# 1202

https://adventofcode.com/2019/day/2

## part 1



In [74]:
import doctest
from typing import Dict, List


class IntcodeProgram():
    """
    
    >>> IntcodeProgram([1,0,0,0,99]).run().program_data == [2,0,0,0,99]
    True
    >>> IntcodeProgram([2,3,0,3,99]).run().program_data == [2,3,0,6,99]
    True
    >>> IntcodeProgram([2,4,4,5,99,0]).run().program_data == [2,4,4,5,99,9801]
    True
    >>> IntcodeProgram([1,1,1,4,99,5,6,0,99]).run().program_data == [30,1,1,4,2,5,6,0,99]
    True
    """
    OPCODE_ADD = 1
    OPCODE_MUL = 2
    OPCODE_END = 99
    OPCODE_DATA = 0
    
    def __init__(self, program_data):
        if isinstance(program_data, str):
            program_data: List[int] = self.program_data_string_to_int_list(program_data)
        self.program_data: List[int] = program_data
        self.validate_program_data()
    
    def __repr__(self):
        return f'IntcodeProgram({self.program_data})'
    
    def __str__(self):
        lines = []
        for opcode, instruction in self.instructions():
            lines.append(instruction)
        return '\n'.join([str(s) for s in lines])
    
    @staticmethod
    def program_data_string_to_int_list(program_data: str) -> List[int]:
        int_list = [int(x.strip()) for x in program_data.split(',')]
        return int_list
    
    def validate_program_data(self) -> bool:
        n_data = len(self.program_data)
        for opcode, instruction in self.instructions():
            if opcode == IntcodeProgram.OPCODE_ADD or opcode == IntcodeProgram.OPCODE_MUL:
                if len(instruction) != 4:
                    raise ValueError(f'Expected the opcode "{opcode}" to have four parts, instead: {instruction}')
                op, address0, address1, address_dest = instruction
                self.validate_address(address0)
                self.validate_address(address1)
                self.validate_address(address_dest)
            if opcode == IntcodeProgram.OPCODE_END:
                if len(instruction) != 1:
                    raise ValueError(f'Expected the opcode "{opcode}" to have one part, instead: {instruction}')
    
    def validate_address(self, address):
        if address >= len(self.program_data):
            raise ValueError(f'Address "{address}" is not valid for a program of length {len(self.program_data)}')
    
    def instructions(self) -> List[int]:
        """Generator for iterating over instructions in `self.program_data`.
        For opcodes that take arguments, it yields (opcode, [opcode, address0, address1, address_dest]).
        For opcodes that do not take arguments, it yields (opcode, [opcode]).
        And for subsequent data section, it yields (opcode, [data0, data1, ...]).
        
        This assumes that the values can be changed in any part of the program data
        between iterations.
        """
        opcode_end_seen = False
        data_iter = iter(self.program_data)
        try:
            while True:
                d = next(data_iter)
                if not opcode_end_seen and (d == IntcodeProgram.OPCODE_ADD or d == IntcodeProgram.OPCODE_MUL):
                    address0 = next(data_iter)
                    address1 = next(data_iter)
                    address_dest = next(data_iter)
                    yield (d, [d, address0, address1, address_dest])
                elif not opcode_end_seen and d == IntcodeProgram.OPCODE_END:
                    opcode_end_seen = True
                    yield (d, [d])
                else:
                    # the rest is data section
                    yield (IntcodeProgram.OPCODE_DATA, [d] + [x for x in data_iter])
        except StopIteration as e:
            pass

    def run(self):
        for opcode, instruction in self.instructions():
            if opcode == IntcodeProgram.OPCODE_ADD:
                op, address0, address1, address_dest = instruction
                val0, val1 = self.program_data[address0], self.program_data[address1]
                self.program_data[address_dest] = val0 + val1
            elif opcode == IntcodeProgram.OPCODE_MUL:
                op, address0, address1, address_dest = instruction
                val0, val1 = self.program_data[address0], self.program_data[address1]
                self.program_data[address_dest] = val0 * val1
        return self
                

# p = IntcodeProgram('1,0,0,0,99')
# print(p)
# p.run()
# print()
# print(p)
doctest.run_docstring_examples(IntcodeProgram, globals(), verbose=True, optionflags=doctest.NORMALIZE_WHITESPACE)

Finding tests in NoName
Trying:
    IntcodeProgram([1,0,0,0,99]).run().program_data == [2,0,0,0,99]
Expecting:
    True
ok
Trying:
    IntcodeProgram([2,3,0,3,99]).run().program_data == [2,3,0,6,99]
Expecting:
    True
ok
Trying:
    IntcodeProgram([2,4,4,5,99,0]).run().program_data == [2,4,4,5,99,9801]
Expecting:
    True
ok
Trying:
    IntcodeProgram([1,1,1,4,99,5,6,0,99]).run().program_data == [30,1,1,4,2,5,6,0,99]
Expecting:
    True
ok


> Once you have a working computer, the first step is to restore the gravity assist program (your puzzle input) to the "1202 program alarm" state it had just before the last computer caught fire. To do this, before running the program, replace position 1 with the value 12 and replace position 2 with the value 2. What value is left at position 0 after the program halts?



In [73]:
puzzle_input = """1,0,0,3,1,1,2,3,1,3,4,3,1,5,0,3,2,6,1,19,1,5,19,23,2,9,23,27,1,6,27,31,1,31,9,35,2,35,10,39,1,5,39,43,2,43,9,47,1,5,47,51,1,51,5,55,1,55,9,59,2,59,13,63,1,63,9,67,1,9,67,71,2,71,10,75,1,75,6,79,2,10,79,83,1,5,83,87,2,87,10,91,1,91,5,95,1,6,95,99,2,99,13,103,1,103,6,107,1,107,5,111,2,6,111,115,1,115,13,119,1,119,2,123,1,5,123,0,99,2,0,14,0"""
p = IntcodeProgram(puzzle_input)
p.program_data[1] = 12
p.program_data[2] = 2
print('Original input (after adjustments)')
print(p)
p.run()
print('Processed data')
print(p)

Original input (after adjustments)
[1, 12, 2, 3]
[1, 1, 2, 3]
[1, 3, 4, 3]
[1, 5, 0, 3]
[2, 6, 1, 19]
[1, 5, 19, 23]
[2, 9, 23, 27]
[1, 6, 27, 31]
[1, 31, 9, 35]
[2, 35, 10, 39]
[1, 5, 39, 43]
[2, 43, 9, 47]
[1, 5, 47, 51]
[1, 51, 5, 55]
[1, 55, 9, 59]
[2, 59, 13, 63]
[1, 63, 9, 67]
[1, 9, 67, 71]
[2, 71, 10, 75]
[1, 75, 6, 79]
[2, 10, 79, 83]
[1, 5, 83, 87]
[2, 87, 10, 91]
[1, 91, 5, 95]
[1, 6, 95, 99]
[2, 99, 13, 103]
[1, 103, 6, 107]
[1, 107, 5, 111]
[2, 6, 111, 115]
[1, 115, 13, 119]
[1, 119, 2, 123]
[1, 5, 123, 0]
[99]
[2, 0, 14, 0]
Processed data
[3101844, 12, 2, 2, 1, 1, 2, 3, 1, 3, 4, 3, 1, 5, 0, 3, 2, 6, 1, 24, 1, 5, 19, 25, 2, 9, 23, 75, 1, 6, 27, 77, 1, 31, 9, 80, 2, 35, 10, 320, 1, 5, 39, 321, 2, 43, 9, 963, 1, 5, 47, 964, 1, 51, 5, 965, 1, 55, 9, 968, 2, 59, 13, 4840, 1, 63, 9, 4843, 1, 9, 67, 4846, 2, 71, 10, 19384, 1, 75, 6, 19386, 2, 10, 79, 77544, 1, 5, 83, 77545, 2, 87, 10, 310180, 1, 91, 5, 310181, 1, 6, 95, 310183, 2, 99, 13, 1550915, 1, 103, 6, 1550917, 1, 107, 5, 

## part 2

> To complete the gravity assist, you need to determine what pair of inputs produces the output 19690720"

In [84]:
# Brute force approach works in reasonable time
for i, j in [(i, j) for i in range(100) for j in range(100)]:
    p = IntcodeProgram(puzzle_input)
    p.program_data[1] = i
    p.program_data[2] = j
    p.run()
    result = p.program_data[0]
    if result == 19690720:
        print(f'({i}, {j}) => {result}')
        print(f'100 * {i} + {j} = {100 * i + j}')

(84, 78) => 19690720
100 * 84 + 78 = 8478
